# Gray-Scott model

The Gray-Scott equations model reaction and diffusion of chemical species $A$ and $B$ which can produce a variety of patterns.
The model is rather simple:

$$
\begin{array}{l}
\frac{\partial A}{\partial t} = D_A \Delta A - AB^2 + f(1-A) \\
\frac{\partial B}{\partial t} = D_B \Delta B + AB^2 - (k+f)B
\end{array}
$$

We can approximate the Laplacian with a 5-points finite difference scheme:

$$
\Delta A_{i,j} \approx A_{i,j-1} + A_{i-1,j} -4A_{i,j} + A_{i+1, j} + A_{i, j+1}
$$

We can approximate the time derivative with an explicit Euler scheme.

## Initialization

We use the following types for vectors and matrices:

In [1]:
#include <vector>

using vector_type = std::vector<double>;
using matrix_type = std::vector<vector_type>;

On the domain $[0, 1]\times[0, 1]$, $A$ holds $1$ and $B$ holds $0$ everywhere except on the square of length $0.2$ centered on the domain. On this square, $A$ holds $0.5$ and $B$ holds $0.25$. 

Write an `init_gray_scott` function that initializes the matrices $A$ and $B$. This function should accept the number of points in each direction (squared grid) in addition to $A$ and $B$.

In [2]:
void init_gray_scott(matrix_type& a, matrix_type&b, size_t nb_points)
{
    a.resize(nb_points, vector_type(nb_points, 1.));
    b.resize(nb_points, vector_type(nb_points, 0.));
    double dx = 1./(nb_points - 1);
    size_t domain_begin = static_cast<size_t>(0.4 / dx), domain_end = 0.6 / dx;
    for (size_t i = domain_begin; i <= domain_end; ++i)
    {
        for (size_t j = domain_begin; j <= domain_end; ++j)
        {
            a[i][j] = 0.5;
            b[i][j] = 0.25;
        }
    }
}

## Boundary conditions

We assume in the following that we have periodic boundary conditions. Write an `apply_boundary_conditions` function which takes a matrix as parameter and applies it periodic boundary conditions.

In [3]:
void apply_boundary_conditions(matrix_type& a)
{
    size_t size = a.size();
    a[0] = a[size - 2];
    a[size - 1] = a[1];
    for (size_t  i = 0; i < size; ++i)
    {
        a[i][0] = a[i][size - 2];
        a[i][size-1] = a[i][1];
    }
}

## Laplacian

Write a `laplacian` function that takes a `(n, n)`matrix and computes its laplacian into a `(n-2, n-2)` matrix. both matrix should be passed as parameters.

In [4]:
void laplacian(const matrix_type& m, matrix_type& res)
{
    size_t size = m.size();
    res.resize(size - 2, vector_type(size - 2));
    
    for(size_t i = 1; i < size - 1; ++i)
    {
        for (size_t j = 1; j < size - 1; ++j)
        {
            res[i-1][j-1] = m[i][j-1] + m[i-1][j] - 4*m[i][j] + m[i+1][j] + m[i][j+1];
        }
    }
}

## Gray-scott

Write the `grayscott` function which takes $A$, $B$, $D_A$, $D_B$, $f$ et $k$ as input parameters and applies the Gray-Scott model. Use an explicit Euler-scheme for the time integration with a step value of $1$. Boundary conditions should be applied after the integration.

In [5]:
void grayscott(matrix_type& a, matrix_type& b, double da, double db, double f, double k)
{
    matrix_type la;
    matrix_type lb;
    laplacian(a, la);
    laplacian(b, lb);
    
    size_t size = a.size() - 2;
    
    for (size_t i = 0; i < size; ++i)
    {
        for (size_t j = 0; j < size; ++j)
        {
            double va = a[i+1][j+1];
            double vb = b[i+1][j+1];
            double abb = va*vb*vb;
            a[i+1][j+1] += da*la[i][j] - abb + f*(1 - va);
            b[i+1][j+1] += db*lb[i][j] + abb - (f + k)*vb;
        }   
    }
    
    apply_boundary_conditions(a);
    apply_boundary_conditions(b);
}

## Simulation

We use the following parameters:

In [6]:
double Da = 0.1, Db = 0.05;
double f = 0.0367, k = 0.0649 ;
// double f = 0.0545, k = 0.062;
// double f = 0.018, k =0.050;
// double f = 0.050, k = 0.065;
// double f = 0.035, k = 0.060;

We will use the following function to dump the results in a file

In [7]:
#include <algorithm>
#include <fstream>
#include <iterator>

void dump_matrix(std::ofstream& out, const matrix_type& m, size_t iteration)
{
    out << iteration << '\n';
    out << m.size() << ',' << m[0].size() << '\n';
    std::for_each(m.cbegin(), m.cend(), [&out](const vector_type& v) {
        std::copy(v.cbegin(), v.cend(), std::ostream_iterator<double>(out, ","));
        out << '\n';
    });
}

Write a `solve_grayscott` function that solves the Gray-Scott equations. Use 20000 time steps for the simulation and 301 points for the space mesh. The function should dump the number of time $steps / 40$ into the `grayscott.txt` file and then one matrix `B` out of 40.

In [8]:
void solve_grayscott()
{
    std::ofstream out("grayscott.txt");
    matrix_type a, b;
    size_t nb_times = 20000;
    size_t nb_spaces = 301;
    
    out << nb_times/40 << '\n';
    init_gray_scott(a, b, nb_spaces);
    for (size_t i = 0; i < nb_times; ++i)
    {
        grayscott(a, b, Da, Db, f, k);
        if (i % 40 == 0)
        {
            dump_matrix(out, b, i/40);
        }
    }
}

In [9]:
solve_grayscott();

Open the `grayscott_visualization` notebook to visualize the results.

In [ ]:
void my_func(double a, double b, double c)
{
}

auto new_func = std::bind(my_func, 3.2, _1, 1.2);
new_func(4.5) <=> my_func(3.2, 4.5, 1.2);

auto new_func2 = std::bind(my_func, _1, 3.2, _2);
new_func(3.3, 4.5) <=> my_func(3.3, 3.2, 4.5);

auto new_func3 = std::bind(my_func, _2, 3.2, _1);
new_func(3.3, 4.5) <=> my_func(4.5, 3.2, 3.3);